# EchoPro Python Workflow <a class="tocSkip">

# Import libraries and configure the Jupyter notebook

In [ ]:
# libraries used in the Notebook
import matplotlib.pyplot as plt

# Python version of EchoPro
import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.semivariogram import SemiVariogram as SV

# Allows us to easily use matplotlib widgets in our Notebook
%matplotlib widget
# %matplotlib inline

# Set up  EchoPro for a specific survey year

## Initialize EchoPro object  using configuration files

* `initialization_config.yml` -- parameters independent of survey year
* `survey_year_2019_config.yml` -- parameters specific to survey year
* `source` -- Define the region of data to use e.g. US, CAN, US & CAN
* `exclude_age1` -- States whether age 1 hake should be included in analysis.

In [ ]:
%%time
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3, 
                             exclude_age1=True)

## Load and process input data 
* This data is stored in `survey_2019`

In [ ]:
%%time 
survey_2019.load_survey_data() #file_type='biological')

In [ ]:
survey_2019.specimen_df.head()

## Compute the areal biomass density
* The areal biomass density is stored in `survey_2019`

In [ ]:
%%time
survey_2019.compute_biomass_density()

In [ ]:
survey_2019.bio_calc.final_biomass_table.head()

# Jolly-Hampton CV Analysis

* Compute the mean of the Jolly-Hampton CV value on data that has not been Kriged
* Note: the algorithm used to compute this value is random in nature

In [ ]:
%%time
CV_JH_mean = survey_2019.run_cv_analysis(kriged_data=False)

In [ ]:
print(f"Mean Jolly-Hampton CV: {CV_JH_mean:.4f}")

# Obtain Kriging Mesh Data

## Access Kriging mesh object
* Reads mesh data files specified by `survey_2019` 

In [ ]:
krig_mesh = survey_2019.get_kriging_mesh()

### Plot the Mesh, Transects and smoothed isobath contour

* Generate interactive map using the Folium package
* Mesh points are in gray
* Transect points are represented by a changing color gradient
* Smoothed contour points (200m isobath) are in blue 

In [ ]:
fmap = krig_mesh.plot_layered_points()
fmap

## Apply coordinate transformations
* Longitude transformation
* Lat/Lon to distance

### Transect points

In [ ]:
krig_mesh.apply_coordinate_transformation(coord_type='transect')

### Mesh points

In [ ]:
krig_mesh.apply_coordinate_transformation(coord_type='mesh')

In [ ]:
# plot the transformed mesh points 
plt.plot(krig_mesh.transformed_mesh_df.x_mesh, 
         krig_mesh.transformed_mesh_df.y_mesh, 'r*', markersize=1.25)
plt.show()

# Compute biomass density Semi-Variogram and fit a model

* Compute the normalized semi-variogram using the areal biomass density
* Fit a model to the semi-variogram values

## Compute the semi-variogram

### Initialize semi-variogram calculation
* Transformed transect points
* Parameters specific to semi-variogram algorithm

In [ ]:
semi_vario = survey_2019.get_semi_variogram(
    krig_mesh,
    params=dict(nlag=30, lag_res=0.002)
)

### Compute the normalized semi-variogram

In [ ]:
%%time
semi_vario.calculate_semi_variogram()

## Fit a model to the semi-variogram

* A widget to easily fit the model

In [ ]:
semi_vario.get_widget()

# Perform Ordinary Kriging of biomass density

* transformed mesh points
* semi-variogram model
* normalized biomass density 

## Initialize Kriging routine

In [ ]:
kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SV.generalized_exp_bessel
)

# uncomment to use widget values 
# kriging_params.update(semi_vario.get_params_for_kriging())

# initalize kriging routine
krig = survey_2019.get_kriging(kriging_params)

## Perform Kriging
* Also generates total biomass at mesh points

In [ ]:
%%time
krig.run_biomass_kriging(krig_mesh)

In [ ]:
krig_results = survey_2019.bio_calc.krig_results_gdf

In [ ]:
print(f"Total Kriged Biomass Estimate: {1e-6*krig_results.krig_biomass_vals.sum():.3f} kmt")

## Plot Kriged Biomass estimate in kmt

In [ ]:
# plot all mesh points
krig_results.krig_biomass_vals = 1e-6 * krig_results.krig_biomass_vals
krig.plot_kriging_results(krig_results, krig_field_name="krig_biomass_vals")